# 5.4 단순한 계층 구현하기

In [8]:
# 곱셈계층
# 아래 계산 그래프 그림넣기 (forward, backward 값 포함)

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# [forward]
apple_price = mul_apple_layer.forward(apple, apple_num) # (1) apple*apple_num
price = mul_tax_layer.forward(apple_price, tax) # (2) apple_price*tax

# [backward]
dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice) # (2)
# 아래 두 개의 연산을 위의 줄에서 실행
# daapple_price(dx, 1.1) = dprice(dout, 1) * tax(y, 1.1)
# dtax(dy, 200) = dprice(dout, 1) * apple_price(x, 200)

dapple, dapple_num = mul_apple_layer.backward(dapple_price) # (1)
# 아래 두 개의 연산을 위의 줄에서 실행
# dapple(dx, 2.2) = dapple_price(dout, 1.1) * apple_num(y, 2)
# dapple_num(dy, 110) = dapple_price(dout, 1.1) * apple(x, 100)

print("dapple_price:", dapple_price)
print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

dapple_price: 1.1
price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


In [9]:
# 덧셈계층
# 아래 계산 그래프 그림넣기 (forward, backward 값 포함) 161p.

class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)

dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
# dapple_price(dx) = dall_price(dout) * 1
# dorange_price(dy) = dall_price(dout) * 1


dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)


price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


# 5.5 활성화 함수 계층 구현하기

In [14]:
# ReLU
import numpy as np


class Relu:
    def __init__(self):
        self.mask = None # input data array 가정

    def forward(self, x):
        self.mask = (x <=0)
        out = x.copy()
        out[self.mask] = 0 
    
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx
    
x = np.array([[1.0, -0.5],
             [-2.0, 3.0]])
print(x, '\n')

mask = (x <=0)
print(mask, '\n')

x[mask] = 0 # True인 원소의 값을 0으로 수정
print(x)

[[ 1.  -0.5]
 [-2.   3. ]] 

[[False  True]
 [ True False]] 

[[1. 0.]
 [0. 3.]]


In [15]:
# sigmoid
# sigmoid 국소 계산 그래프 그림 -> 간소화 y(1-y) 도출과정

class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1/(1+ np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

# 5.6 Affine/Softmax 계층 구현

In [17]:
# affine transformation 개념
# affine 계산그래프 그림
# w.t, x.t 혹은 곱 순서 변경 why?
# -> 원래 shape과 backward gradient shape 동일하도록 곱을 재구성

X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

X.shape # (2,)
W.shape # (2,3)
B.shape # (3,)

Y = np.dot(X, W) + B
print(X.shape, '<dot>', W.shape)
print(Y.shape) # (3,)

(2,) <dot> (2, 3)
(3,)


In [18]:
# X 한개가 아닌 Batch개의 data라면?

# [batch_forward]
X_dot_W = np.array([[0,0,0], # 2개의 forward output인 Y(3,)
                   [10,10,10]])
B = np.array([1,2,3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [19]:
X_dot_W + B # Bias 합에 broadcasting

array([[ 1,  2,  3],
       [11, 12, 13]])

In [20]:
dY = np.array([[1,2,3],
              [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [21]:
dB = np.sum(dY, axis =0)
dB

array([5, 7, 9])

In [22]:
# [Bias_backward]
dY = np.array([[1,2,3], # 2개 data 대한 dY
              [4,5,6]])
print(dY)

dB = np.sum(dY, axis = 0) # 각 데이터 dY를 합 -> dB
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        
        # 3d 이상의 tensor input가능
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        # 원래 x shape으로
        dx = dx.reshape(*self.original_x_shape)  
        return dx


In [ ]:
# softmax loss layer 구현
# softmax 계산그래프 그림과 설명
# cross entropy backward값의 의미
# softmax - label 간 error 차이 예시 (good_pred, bad_pred)

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx